In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.cloud import bigquery


In [3]:
spark = SparkSession.builder.appName('geoEDA').getOrCreate()
    #.config('spark.jars, "gs://spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.23.2.jar"')\
    
    

#conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','4g')])

spark.sparkContext.getConf().getAll()

[('spark.stage.maxConsecutiveAttempts', '10'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.submit.pyFiles',
  '/root/.ivy2/jars/com.johnsnowlabs.nlp_spark-nlp_2.12-4.4.0.jar,/root/.ivy2/jars/graphframes_graphframes-0.8.2-spark3.1-s_2.12.jar,/root/.ivy2/jars/com.typesafe_config-1.4.2.jar,/root/.ivy2/jars/org.rocksdb_rocksdbjni-6.29.5.jar,/root/.ivy2/jars/com.amazonaws_aws-java-sdk-bundle-1.11.828.jar,/root/.ivy2/jars/com.github.universal-automata_liblevenshtein-3.0.0.jar,/root/.ivy2/jars/com.google.cloud_google-cloud-storage-2.16.0.jar,/root/.ivy2/jars/com.navigamez_greex-1.0.jar,/root/.ivy2/jars/com.johnsnowlabs.nlp_tensorflow-cpu_2.12-0.4.4.jar,/root/.ivy2/jars/it.unimi.dsi_fastutil-7.0.12.jar,/root/.ivy2/jars/org.projectlombok_lombok-1.16.8.jar,/root/.ivy2/jars/com.google.guava_guava-31.1-jre.jar,/root/.ivy2/jars/com.google.guava_failureaccess-1.0.1.jar,/root/.ivy2/jars/com.google.guava_listenablefuture-9999.0-empty-to-avoid-conflict-

In [4]:
bq_client = bigquery.Client(project='msca-bdp-student-ap')


In [6]:
bucket = "msca-bdp-student-gcs"
spark.conf.set('temporaryGcsBucket', bucket)

# Load data from BigQuery.
df = spark.read.format('bigquery') \
  .option('table', 'msca-bdp-student-ap.chicago_rideshare.community_areas') \
  .load()
df.createOrReplaceTempView('community')

df2 = spark.sql(
    'SELECT * FROM community LIMIT 5')
df2.show()
df2.printSchema()


+-------------+-------+----------+----------+----------+---------+----+--------------------+---------------+---------------+
|    shape_len|comarea|comarea_id|area_numbe|area_num_1|perimeter|area|            geometry|     shape_area|      community|
+-------------+-------+----------+----------+----------+---------+----+--------------------+---------------+---------------+
|31027.0545098|      0|         0|        35|        35|        0|   0|POLYGON((-87.6093...|4.60046211581E7|        DOUGLAS|
|19565.5061533|      0|         0|        36|        36|        0|   0|POLYGON((-87.5921...|1.69139610408E7|        OAKLAND|
|25339.0897503|      0|         0|        37|        37|        0|   0|POLYGON((-87.6288...|1.99167048692E7|    FULLER PARK|
|28196.8371573|      0|         0|        38|        38|        0|   0|POLYGON((-87.6067...|4.84925031554E7|GRAND BOULEVARD|
|23325.1679062|      0|         0|        39|        39|        0|   0|POLYGON((-87.5923...|2.90717419283E7|        KENWOOD|


In [10]:
%%bigquery df

SELECT 
    count(*) as num_rides,
    community_areas.community
    FROM `chicago_rideshare.community_areas` as community_areas
    INNER JOIN `chicago_rideshare.geo_program_area_2020` as ride_locations 
    ON (ST_CONTAINS(community_areas.geometry, ride_locations.pickup_geo))
    GROUP BY community_areas.community
    ORDER BY num_rides desc

Query is running:   0%|          |

Downloading:   0%|          |

In [12]:
df.head(20)

,num_rides,community
0,221531,HYDE PARK
1,42102,WOODLAWN
2,32421,KENWOOD


In [21]:
%%bigquery df2

CREATE OR REPLACE TABLE chicago_rideshare.temp AS
SELECT
  rides.*,
  areas.community, areas.area_num_1, areas.geometry
FROM
  chicago_rideshare.geo_program_area_2020 rides
LEFT JOIN
  chicago_rideshare.community_areas areas
ON
  ST_WITHIN(ST_GEOGPOINT(rides.pickup_lon, rides.pickup_lat), areas.geometry) OR
  ST_WITHIN(ST_GEOGPOINT(rides.dropoff_lon, rides.dropoff_lat), areas.geometry)


Query is running:   0%|          |

In [20]:
df2.head(20)

""
